# **This Notebook is used for the third assignment**

## *This is the first part of the assignment*

### **Introduction**

I'm an employee of a foreign development and investment firm that is seeking different opportunities in Toronto area. There is no limit to what the firm invests as long as it's profitable opportunity. We are looking to enter the Toronto market because it is a big city and there's a lot of traffic.

However, with us being a foreign company and had very little local knowledge, we need to do some exploratory analysis of the city before we decide on our entry strategy and projects. I'm tasked to do some research to have a high-level understanding of the metropolitan area and present the findings to my supervisor and investors.

### **Business Problem**

In this analysis, I'm trying to get a high level understanding of the city. Since we are hoping to invest in various types of opportunities, we will be looking at different venues in different neighborhoods. There are several neighborhoods in Toronto area. It will be less efficient to exaxmine each neighborhood for a high-level analysis like this. Therefore, I'm planning to cluster these neighborhoods into clusters based on their venue counts. After determining the optimal number of clusters and successfully clustering neighborhoods, I can check within each cluster to see what are the majority of the venues in each cluster. The result of this observation can give me a high-level understanding of each cluster, namely, I can say if cluster/district 1 is a food district and district 2 is a financial district, etc.

To summarize, my business problem is to determine the optimal number of clusters of neighborhoods in Toronto, and then to hopefully examine each cluster to determine what kind of district it is to inform our investment strategy.

## *This is the second part of the assignment*

### **Data Collection**
In order to solve my business problem, I'll need geological information for all neighborhoods in Toronto, especially latitudes and longitudes. We'll also need the categories of the venues to determine the type of district they are located in. For example, if a cluster has a lot of venues categorized as restaurant or coffee shop, we can say this cluster is more likely a food district.

Fortunately, all the data can be obtained from the internet with a quick parse and download. The venue data can be obtained from the Foursquare data center. We don't need additional data sources. The data collection process is shown below.

Import libraries

In [11]:
!pip install beautifulsoup4 lxml html5lib
import urllib.request
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup


Import data from last module's source and put it in a data frame

In [14]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html=urllib.request.urlopen(url)
soup=BeautifulSoup(html,'html.parser')
all_tables=soup.find_all("table")
right_table=soup.find('table', class_='wikitable sortable')
A=[]
B=[]
C=[]
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

df=pd.DataFrame(A, columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C
df.head()

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


We see that there is a "\n" at the end of each data as a result of parsing. Get rid of that.

In [15]:
df['Postal Code']=df['Postal Code'].str[:-1]
df['Borough']=df['Borough'].str[:-1]
df['Neighborhood']=df['Neighborhood'].str[:-1]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Now get rid of boroughs with values of "not assigned" 

In [17]:
df=df[df['Borough']!='Not assigned'].reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


We also need the latitudes and longitudes of each neighborhood

In [18]:
url2='http://cocl.us/Geospatial_data'
df2=pd.read_csv(url2)
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Combine it with the first dataframe for the final dataframe. This is the geological part of the dataset**

In [23]:
tt=df.join(df2.set_index('Postal Code'), on=('Postal Code'))
tt.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Now that we have the neighborhoods ready, we need the venues from these neighborhoods

In [20]:
import json
import requests
from pandas.io.json import json_normalize

In [21]:
CLIENT_ID = 'NMFYBF4VNTHW1TTTHZAGOUEWZFYOAGSWGOENRF5FMOZJUTKP' # your Foursquare ID
CLIENT_SECRET = 'VKNWMMPBFWO2YPGDVJHSECMXHPX4ZSYP5MMOY0GBEIEA2C4O' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NMFYBF4VNTHW1TTTHZAGOUEWZFYOAGSWGOENRF5FMOZJUTKP
CLIENT_SECRET:VKNWMMPBFWO2YPGDVJHSECMXHPX4ZSYP5MMOY0GBEIEA2C4O


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url1).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Use the above function to get the venues near each neighborhood

In [24]:
tt_venues=getNearbyVenues(names=tt['Neighborhood'], latitudes=tt['Latitude'], longitudes=tt['Longitude'], radius=500)
tt_venues.head()

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [25]:
tt_venues.shape

(2136, 7)

let's see how many venues are in each neighborhood

In [27]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
tt_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Berczy Park,55,55,55,55,55,55
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16


## **This concludes the third assignment**